# 1 Feature Calculation
To train a ML using chemical structures you need to translate the chemical structure into numbers.  
There are different ways:
- Calculation of descriptors (Different characteristics of the molecule)
- Calculation of fingerprints (Different structural features are OneHotEncoded to 0 and 1.)  

For both ways, there are different algorithms and software to calculate.  
- RDKit
- PaDEL  

The PaDEL is build with java, but there is a python wrapper to use is in python for calculation -> [padelpy](https://github.com/ecrl/padelpy)  
This is what I used for my project.

For the beginning I will start with the standard in padelpy -> *PubChem Fingerprints* and standard PaDEL *Descriptors*

In [ ]:
# Install padelpy if not excist
#%pip install padelpy

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from padelpy import from_sdf
import glob

## 1.2 Calculation of the desriptors using PaDEL and .sdf files of the molecules

As input for the feature calculation there are different options. One is using the SMILES notation of a molecule or using the Structures Data File (SDF). The last one is the output of the chemical database [PubChem](https://pubchem.ncbi.nlm.nih.gov/). Since the data from the underlying paper was available as SDF files I used the `from_sdf()` function from the `padelpy` wrapper.  
Someone can use the `pubchempy` API ([docs](https://pubchempy.readthedocs.io/en/latest/)) to interact with the PubChem database to get information and SDF files for certain chemicals.  


In [2]:
# read all data from the paper
sdf_files = glob.glob("./data/sdf/*.sdf")
sdf_files.sort()
sdf_files

['./data/sdf/Negative_Test_upload.sdf',
 './data/sdf/Positive_Test_upload.sdf',
 './data/sdf/Real_Data.sdf',
 './data/sdf/Train_Negative.sdf',
 './data/sdf/Train_positive.sdf',
 './data/sdf/test_molecule.sdf']

In [3]:
# Create container to store the calculated descriptors
descriptors_molib = []
not_calculated = []

In [4]:
# loop through the files and calculate descriptors
for i, file in enumerate(sdf_files[0:2]):
    try:
        descriptors_molib += from_sdf(file, threads=2)
        descriptors_molib[-1]["dataset"] = file.split("/")[-1][:-4]
    except:
        print(f"{file} coudnt be calculated")
        not_calculated += [file]
        continue
    print(file.split("/")[-1][:-4])

./data/sdf/Negative_Test_upload.sdf coudnt be calculated


In [ ]:
# create a DF from descriptors and safe as CSV
descriptors_molib_df = pd.DataFrame(descriptors_molib)
descriptors_molib_df.to_csv("/data/descriptors_molib_df.csv", index=False)

# create df for the molecules which weren't calculated
not_calculated_df = pd.DataFrame(not_calculated)
# create better names
not_calculated_df = not_calculated_df.iloc[:, 0].str.split("/", expand=True).loc[:3]
not_calculated_df.to_csv("/data/desc_not_calc.csv", index=False)

It is possible to use a SDF file, with all molecules in one file, if the molecules are seperated by `$$$$`.  
Not working to calculate because of *timeout-error*:
- `Negative_Test_upload.sdf`
- `Positive_Test_upload.sdf`
- `Real_Data.sdf`
- `Train_Negative.sdf`
- `Train_Positive.sdf`

Since in every file there was at least one molecule, where it wasn't possible to calculate the descriptors, there was a need to split the SDF file into single SDF-files for each molecule.

## 1.3 Split the SDF files into single SDF files for each molecule

The script i wrote for splitting can be found [here](./sdf_split.py)

In [ ]:
# importing the script to split the SDF files
from sdf_split import split_into_files

# Splitting the sdf file with all structural information of all molecules
for file in sdf_files:
    split_into_files(file, "./data/sdf/output")

In [ ]:
# read all produced sdf files
sdf_files = glob.glob("./data/sdf/output/*.sdf")
sdf_files.sort()
sdf_files

## 1.4 Calculation of Fingerprints (PubChem)

In [ ]:
fp_molib = []
not_calculated_fp = []

In [ ]:
for i, file in enumerate(sdf_files):
    try:
        fp_molib += from_sdf(file,
                             fingerprints=True,
                             descriptors=False,
                             threads=-1)
        fp_molib[-1]["dataset"] = file.split("/")[-1][:-4]
    except:
        print(f"{file} coudnt be calculated")
        not_calculated_fp += [file]
        continue
    print(file.split("/")[-1][:-4])

In [ ]:
fp_molib_df = pd.DataFrame(fp_molib)

In [ ]:
fp_molib_df.head()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,dataset
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Negative_Test_upload001
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Negative_Test_upload002
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Negative_Test_upload003
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Negative_Test_upload004
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Negative_Test_upload005


In [ ]:
fp_molib_df.to_csv("/data/fp_molib_df.csv", index=False)

## 1.5 Calculation of descriptors II

In [ ]:
# Create container to store the calculated descriptors
descriptors_molib = []
not_calculated = []

In [ ]:
# looping the the sdf files
for i, file in enumerate(sdf_files):
    try:
        
        descriptors_molib += from_sdf(file,
                                      fingerprints=True,
                                      descriptors=False,
                                      threads=-1)
        # Creating a column where the descriptors come from
        descriptors_molib[-1]["dataset"] = file.split("/")[-1][:-4]
    except:
        print(f"{file} coudnt be calculated")
        not_calculated += [file]
        continue
    # checking the progress
    print(file.split("/")[-1][:-4])

## 1.6 Calculate other FP
There are alot of other algorithms to calculate molecule fingerprints. In PaDEL there are 12 different versions. I decided to test two of those, which i found performing well in some paper.
- CDK-extended
- MACCS

Therefore i use the `padeldescriptor()` function, where i have more control over the calculation.

In [4]:
# Create a list with all the fingerprints
FP_list = [
    #'AtomPairs2DCount',
    # 'AtomPairs2D',
    # 'EState',
    'CDKextended',
    # 'CDK',
    # 'CDKgraphonly',
    # 'KlekotaRothCount',
    # 'KlekotaRoth',
    'MACCS' # ,
    # 'PubChem',
    # 'SubstructureCount',
    # 'Substructure'
    ]

In [ ]:
# these files are used to assign the fingerprint in the function
xml_files = glob.glob("/data/xml/*.xml")
xml_files.sort()
xml_files

In [6]:
# creating a dictionary with the names of the calculation as key and the path to the XML file as values
fp_dict = dict(zip(FP_list, xml_files))

In [2]:
# import the function
from padelpy import padeldescriptor

##### Maybe for the fingerprints the sdf-files with all molecules in one file could be possible -> would be easier and maybe faster

In [ ]:
fp_list = ["CDKextended", "MACCS"]

# looping through the FP list
for fp in fp_list:
    descriptor_type = fp_dict[fp]

    # Dataframe for saving the resulting FP
    fp_molib_df = pd.DataFrame()

    # looping through the sdf files
    for i, file in enumerate(sdf_files):
        print(f"{fp}, {i}/{len(sdf_files)}")
        fingerprint_output = f"/data/{fp}_molib_{i}.csv"

        padeldescriptor(mol_dir=file,
                        d_file=fingerprint_output,
                        descriptortypes=descriptor_type,
                        detectaromaticity=True,
                        standardizenitro=True,
                        standardizetautomers=True,
                        threads=-1,
                        removesalt=True,
                        log=True,
                        fingerprints=True
                        )

        # Reading the produced .csv file and add to the dataframe
        temp_df = pd.read_csv(f"/data/{fp}_molib_{i}.csv")
        # adding what dataset the molecule belongs
        temp_df["dataset"] = file.split("/")[-1][:-7]
        fp_molib_df = pd.concat([fp_molib_df, temp_df], ignore_index=True)

    fp_molib_df.drop(columns="Name", inplace=True)
    fp_molib_df.to_csv(f"/data/fp_{fp}_molib_df.csv", index=False)


After saving all calculated descriptors and fingerprints, the features can be used to train the model.